##Do not use run all if you want to then remove the last cell
# 🟣GLM-4.7 Flash × Claude Opus 4.5 XHigh Reasoning Distill

---

### 🧠 What is this?

A **ready-to-run** Google Colab notebook that loads and chats with GLM-4.7-Flash-Claude-Opus-4.5-High-Reasoning-Distill from this [**Repository**](https://huggingface.co/TeichAI/GLM-4.7-Flash-Claude-Opus-4.5-High-Reasoning-Distill-GGUF) — This Model is a quantization of a finetune of a finetune of GLM 4.7 Flash.

> **TL;DR** — fine tune of GLM 4.7 flash with finetuning using opus 4.5 xhigh dataset

### 🏗️ Architecture

```
GLM-4.7 Flash (MoE 64×2.6B)
    ↓ fine-tuned on
Claude Opus 4.5 High Reasoning dataset (250 examples)
    ↓ quantized to
GGUF (Q4_K_M / IQ4_XS / Q3_K_M)
    ↓ runs on
llama.cpp via llama-cpp-python (CUDA + CPU)
    ↓ served through
Gradio Chat UI with streaming
```

---

### 📋 How to use

1. **Run cells 1→4 in order** (takes ~5 min first time for download)
2. **Cell 2** — pick your model quant from the dropdown depending on what you want.
3. **Cell 3** — adjust GPU layers slider (start at 48, lower if OOM)
4. **Cell 4** — launches Gradio chat, click the **public URL** to open or chat in output itself
5. **Cell 5** — run this only if you want to use a different model without having to disconnect and delete runtime this to fully unload model before switching quants this saves time

---

### 🔧 Default Sampling Parameters

*From the [model card](https://huggingface.co/TeichAI/GLM-4.7-Flash-Claude-Opus-4.5-High-Reasoning-Distill-GGUF):*

| Setting | General use | Tool-calling |
|---|---|---|
| Temperature | `1.0` | `0.7` |
| Top-P | `0.95` | `1.0` |
| Min-P | `0.01` | `0.01` |
| Repeat penalty | `1.0` (disabled) | `1.0` (disabled) |

---

### 💾 VRAM Requirements

| Quant | File size | 48 layers GPU | Recommended split (T4 15GB) |
|---|---|---|---|
| **IQ2_M to Q3_K_M** | model<15 GB | ✅ fits fully | `48` layers GPU |
| **IQ4_XS to Q6_K** | 26Gb>model>15Gb | 🪓 needs split | Variable depending on which model youre using |
| **Q8_0 to F16** | model>26 GB | ❌ Model cannot fit on T4 with 12GB ram | Switch to a higher tier |

---

### 📜 Credits

- **Base model**: [GLM-4.7-Flash](https://huggingface.co/zai-org/GLM-4.7-Flash) by zai-org
- **Fine-tune 1**: [Unsloth](https://huggingface.co/unsloth/GLM-4.7-Flash)
- **Fine-tune 2**: [TeichAI](https://huggingface.co/TeichAI/GLM-4.7-Flash-Claude-Opus-4.5-High-Reasoning-Distill) using Claude Opus 4.5 xhigh reasoning [**dataset**](https://huggingface.co/datasets/TeichAI/claude-4.5-opus-high-reasoning-250x)
- **GGUF quantization**: [**TeichAI**](https://huggingface.co/TeichAI/GLM-4.7-Flash-Claude-Opus-4.5-High-Reasoning-Distill-GGUF)
- I built this notebook using AI I dont know how to code if you have any issues blame the AI or Let me know in some way so I can update it (I dont even know how to use github)
I want to write the specific model name also but idk if thats allowed.

In [ ]:
# ============================================================
# @title 📦 CELL 1 — Install Dependencies
# ============================================================

import os
os.environ["PIP_NO_WARN_SCRIPT_LOCATION"] = "1"

!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 2>&1 | grep -v "already satisfied" | tail -1
!pip install huggingface_hub gradio Pillow psutil 2>&1 | grep -v "already satisfied" | tail -1

# Suppress the "restart runtime" warning — not needed here
import warnings
warnings.filterwarnings("ignore", message=".*previously imported.*")
warnings.filterwarnings("ignore", message=".*restart.*")

from IPython.display import clear_output
clear_output()
print("✅ Dependencies installed")

In [ ]:
# ============================================================
# @title 📥 CELL 2 — Download GGUF Model
# ============================================================

#@markdown ### Choose your quant:
MODEL_CHOICE = "Q3_K_M ~ 14.4GB" #@param ["IQ2_M ~ 9.9GB", "IQ3_XS ~ 12.3GB", "Q3_K_S ~ 13GB", "IQ3_M ~ 13.2GB", "Q3_K_M ~ 14.4GB", "IQ4_XS ~ 16GB (43 GPU layers)", "IQ4_NL ~ 17GB", "Q4_K_M ~ 18.1GB", "Q5_K_M ~ 21.3GB", "Q6_K ~ 24.6GB (28 GPU layers)", "Q8_0 ~ 31.8GB", "BF16 ~ 59.9GB", "F16 ~ 59.9GB"]
#@markdown <sub>IQ2_M → Q3_K_M fit fully on free T4 GPU | IQ4_XS → Q6_K need GPU+CPU split which I have noticed gives around 6tps at the start  (Adjust the GPU layer Slider to split the model) | Q8_0+ need mmap (Disk retrevial) or more VRAM (Higher memory machines)</sub>

from huggingface_hub import hf_hub_download
import os
from IPython.display import clear_output

MODEL_REPO = "TeichAI/GLM-4.7-Flash-Claude-Opus-4.5-High-Reasoning-Distill-GGUF"

# Map selection to filename
_model_map = {
    "IQ2_M ~ 9.9GB":   "glm-4.7-flash-claude-4.5-opus.iq2_m.gguf",
    "IQ3_XS ~ 12.3GB": "glm-4.7-flash-claude-4.5-opus.iq3_xs.gguf",
    "Q3_K_S ~ 13GB":   "glm-4.7-flash-claude-4.5-opus.q3_k_s.gguf",
    "IQ3_M ~ 13.2GB":  "glm-4.7-flash-claude-4.5-opus.iq3_m.gguf",
    "Q3_K_M ~ 14.4GB": "glm-4.7-flash-claude-4.5-opus.q3_k_m.gguf",
    "IQ4_XS ~ 16GB (43 GPU layers)":   "glm-4.7-flash-claude-4.5-opus.iq4_xs.gguf",
    "IQ4_NL ~ 17GB":   "glm-4.7-flash-claude-4.5-opus.iq4_nl.gguf",
    "Q4_K_M ~ 18.1GB": "glm-4.7-flash-claude-4.5-opus.q4_k_m.gguf",
    "Q5_K_M ~ 21.3GB": "glm-4.7-flash-claude-4.5-opus.q5_k_m.gguf",
    "Q6_K ~ 24.6GB (28 GPU layers)":   "glm-4.7-flash-claude-4.5-opus.q6_k.gguf",
    "Q8_0 ~ 31.8GB":   "glm-4.7-flash-claude-4.5-opus.q8_0.gguf",
    "BF16 ~ 59.9GB":   "glm-4.7-flash-claude-4.5-opus.bf16.gguf",
    "F16 ~ 59.9GB":    "glm-4.7-flash-claude-4.5-opus.f16.gguf",
}

MODEL_FILE = _model_map[MODEL_CHOICE]
SAVE_DIR   = "/content/models"

os.makedirs(SAVE_DIR, exist_ok=True)

# Clean up old quants to free disk
for old in os.listdir(SAVE_DIR):
    if old.endswith(".gguf") and old != MODEL_FILE:
        old_path = os.path.join(SAVE_DIR, old)
        old_size = os.path.getsize(old_path) / 1e9
        os.remove(old_path)
        print(f"🗑️ Removed: {old} ({old_size:.1f} GB)")

print(f"⬇️ Downloading {MODEL_FILE}...")

model_path = hf_hub_download(
    repo_id=MODEL_REPO,
    filename=MODEL_FILE,
    local_dir=SAVE_DIR,
    token=False,
)

clear_output()
size_gb = os.path.getsize(model_path) / 1e9
quant_tag = MODEL_CHOICE.split(" ~")[0]
print(f"✅ Downloaded — {size_gb:.2f} GB ({quant_tag})")
print(f"📁 {model_path}")

# Store for Cell 3
LOADED_MODEL_PATH = model_path
LOADED_QUANT_TAG = quant_tag

In [ ]:
#@title Keep Alive for Mobile Users (Optional Cell) (I dont even know if this does anything My phone needs 0.001 second to remove page from mem)
from IPython.display import Audio, display
display(Audio("https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a", autoplay=True))

In [ ]:
# ============================================================
# @title 🧠 CELL 3 — Load Model
# ============================================================

#@markdown ### GPU layers (48 = all):
GPU_LAYERS = 48 #@param {type:"slider", min:0, max:48, step:1}


CONTEXT_LENGTH = 4096

from llama_cpp import Llama
from IPython.display import clear_output
import subprocess, sys, io, psutil

print(f"📊 Loading {LOADED_QUANT_TAG} with {GPU_LAYERS}/48 layers on GPU, ctx={CONTEXT_LENGTH}\n")

# Show memory before
ram_before = psutil.virtual_memory().used / 1e9
try:
    smi = subprocess.check_output(
        ["nvidia-smi", "--query-gpu=memory.free", "--format=csv,noheader,nounits"],
        text=True
    ).strip()
    print(f"   VRAM free: {int(smi)} MiB | RAM used: {ram_before:.1f} GB")
except:
    pass

print(f"   ⏳ This may take a minute...\n")

_stderr = sys.stderr
sys.stderr = io.StringIO()

try:
    llm = Llama(
        model_path=LOADED_MODEL_PATH,
        n_gpu_layers=GPU_LAYERS,
        n_ctx=CONTEXT_LENGTH,
        n_batch=384,
        use_mmap=False,
        use_mlock=False,
        verbose=True,
    )
    load_ok = True
except (ValueError, RuntimeError) as e:
    sys.stderr = _stderr
    load_ok = False
    print(f"❌ Failed with {GPU_LAYERS} layers + ctx={CONTEXT_LENGTH} — OOM")
    print(f"   Try reducing GPU_LAYERS or CONTEXT_LENGTH and re-run this cell")
    llm = None

if load_ok:
    sys.stderr = _stderr
    clear_output()
    ram_after = psutil.virtual_memory().used / 1e9
    cpu_layers = 48 - GPU_LAYERS
    try:
        smi = subprocess.check_output(
            ["nvidia-smi", "--query-gpu=memory.used,memory.free",
             "--format=csv,noheader,nounits"], text=True
        ).strip().split(", ")
        print(f"✅ G4.7F·OXH loaded ({LOADED_QUANT_TAG})")
        print(f"   🎮 VRAM: {int(smi[0])} MiB used | {smi[1]} MiB free")
        print(f"   🧠 RAM:  {ram_after:.1f} GB (+{ram_after - ram_before:.1f} GB for CPU layers)")
        print(f"   📊 {GPU_LAYERS} layers GPU + {cpu_layers} layers CPU | ctx={CONTEXT_LENGTH}")
    except:
        print(f"✅ G4.7F·OXH loaded — {GPU_LAYERS} GPU + {cpu_layers} CPU | ctx={CONTEXT_LENGTH} ({LOADED_QUANT_TAG})")

In [ ]:
# ==========================================================================================
# @title 💬 CELL 4 — Gradio Chat UI (Click the link in the output to open Gradio in new tab)
# ==========================================================================================

import gradio as gr
import time, sys, io, re, json, uuid, os, gc
from datetime import datetime
import PIL.Image, PIL.ImageDraw, PIL.ImageFont

MODEL_LABEL = "G4.7F·OXH"

# ── Default settings ──
DEFAULT_SYSTEM_PROMPT = "You are a helpful assistant." #@param {type:"string"}
DEFAULT_MAX_TOKENS = 1024
DEFAULT_TEMPERATURE = 1.0
DEFAULT_TOP_P = 0.95
DEFAULT_MIN_P = 0.01
DEFAULT_REPEAT_PENALTY = 1.0
DEFAULT_FONT_SIZE = 14
DEFAULT_UI_SCALE = 100
#@markdown To change more parameters use settings in gradio UI


# ══════════════════════════════════════════════════════════════
# Thinking Extraction & Formatting
# ══════════════════════════════════════════════════════════════

def extract_thinking(text):
    has_close = "</think>" in text
    has_open = "<think>" in text
    if has_close and not has_open:
        parts = text.split("</think>", 1)
        thinking = parts[0].strip()
        clean = parts[1].strip() if len(parts) > 1 else ""
        if thinking and clean:
            return thinking, clean
        elif thinking and not clean:
            return None, thinking
        else:
            return None, text.strip()
    if has_open and has_close:
        m = re.search(r"<think>(.*?)</think>", text, re.DOTALL)
        if m:
            thinking = m.group(1).strip()
            clean = (text[:m.start()] + text[m.end():]).strip()
            if thinking and clean:
                return thinking, clean
    return None, text.strip()


def format_streaming(full_text, token_count, elapsed):
    tok_s = token_count / elapsed if elapsed > 0 else 0
    stats = f"\n\n<sub>⏳ {token_count} tok · {tok_s:.1f} t/s</sub>"
    if "</think>" in full_text:
        thinking, clean = extract_thinking(full_text)
        display = ""
        if thinking:
            display += (
                f"<details><summary>💭 <i>Thinking</i> ({len(thinking)} chars)"
                f" — click to expand</summary>\n\n{thinking}\n\n</details>\n\n"
            )
        display += clean + stats
        return display
    else:
        if token_count < 3:
            return f"💭 *Thinking...*" + stats
        else:
            return f"💭 **Thinking...**\n\n{full_text}" + stats


def format_final(full_text, token_count, elapsed):
    tok_s = token_count / elapsed if elapsed > 0 else 0
    thinking, clean = extract_thinking(full_text)
    display = ""
    if thinking:
        display += (
            f"<details><summary>💭 <i>Thinking</i> ({len(thinking)} chars)"
            f" — click to expand</summary>\n\n{thinking}\n\n</details>\n\n"
        )
    display += clean
    display += f"\n\n<sub>📊 {token_count} tok · {tok_s:.1f} t/s · {elapsed:.1f}s</sub>"
    return display


def clean_history_text(bot_msg):
    if not bot_msg:
        return ""
    raw = re.sub(r"<details>.*?</details>", "", bot_msg, flags=re.DOTALL)
    raw = re.sub(r"<sub>.*?</sub>", "", raw, flags=re.DOTALL)
    raw = re.sub(r"💭 \*\*?Thinking\.\.\.?\*\*?.*?\n\n", "", raw, flags=re.DOTALL)
    return raw.strip()


# ══════════════════════════════════════════════════════════════
# File/Image Processing
# ══════════════════════════════════════════════════════════════

SUPPORTED_TEXT_EXT = {".txt", ".md", ".py", ".js", ".ts", ".html", ".css",
                     ".json", ".csv", ".xml", ".yaml", ".yml", ".log",
                     ".sh", ".bat", ".sql", ".c", ".cpp", ".h", ".java",
                     ".rs", ".go", ".rb", ".php", ".r", ".tex", ".ini",
                     ".cfg", ".toml", ".env"}

SUPPORTED_IMG_EXT = {".png", ".jpg", ".jpeg", ".gif", ".bmp", ".webp"}


def process_uploaded_file(file_obj):
    if file_obj is None:
        return None, None, None
    filepath = file_obj.name if hasattr(file_obj, 'name') else str(file_obj)
    filename = os.path.basename(filepath)
    ext = os.path.splitext(filename)[1].lower()

    if ext in SUPPORTED_TEXT_EXT:
        try:
            with open(filepath, "r", encoding="utf-8", errors="replace") as f:
                content = f.read()
            if len(content) > 15000:
                content = content[:15000] + f"\n\n... [truncated — file was {len(content)} chars]"
            return content, "text", filename
        except:
            return None, "unsupported", filename

    if ext == ".csv":
        try:
            with open(filepath, "r", encoding="utf-8", errors="replace") as f:
                content = f.read()
            if len(content) > 15000:
                content = content[:15000] + f"\n\n... [truncated]"
            return content, "text", filename
        except:
            return None, "unsupported", filename

    if ext in SUPPORTED_IMG_EXT:
        return filepath, "image", filename

    if ext == ".pdf":
        try:
            import subprocess
            result = subprocess.run(
                ["pdftotext", filepath, "-"],
                capture_output=True, text=True, timeout=30
            )
            if result.returncode == 0 and result.stdout.strip():
                content = result.stdout.strip()
                if len(content) > 15000:
                    content = content[:15000] + f"\n\n... [truncated]"
                return content, "text", filename
        except:
            pass
        return None, "unsupported", filename

    return None, "unsupported", filename


# ══════════════════════════════════════════════════════════════
# Chat Store Helpers
# ══════════════════════════════════════════════════════════════

def make_chat(name=None):
    cid = str(uuid.uuid4())[:8]
    data = {
        "name": name or "New Chat",
        "history": [],
        "created": datetime.now().strftime("%m/%d %H:%M"),
    }
    return cid, data


def chat_label(data):
    return f"{data['name']}  ·  {data['created']}"


def radio_choices(store, search=""):
    items = []
    for cid, data in store.items():
        if not search or search.lower() in data["name"].lower():
            items.append((chat_label(data), data["created"], cid))
    items.sort(key=lambda x: x[1], reverse=True)
    return [x[0] for x in items]


def label_to_id(store, label):
    for cid, data in store.items():
        if chat_label(data) == label:
            return cid
    return None


_init_id, _init_data = make_chat()
_init_store = {_init_id: _init_data}


# ══════════════════════════════════════════════════════════════
# Sidebar Event Handlers
# ══════════════════════════════════════════════════════════════

def on_new_chat(store, active, history):
    if active in store:
        store[active]["history"] = list(history)
    cid, data = make_chat()
    n = sum(1 for d in store.values() if d["name"].startswith("New Chat"))
    if n > 0:
        data["name"] = f"New Chat {n + 1}"
    store[cid] = data
    choices = radio_choices(store)
    lbl = chat_label(data)
    return store, cid, [], gr.update(choices=choices, value=lbl), ""


def on_select_chat(selected, store, active, history):
    if not selected:
        return active, history, store, ""
    if active in store:
        store[active]["history"] = list(history)
    new_id = label_to_id(store, selected)
    if new_id and new_id in store:
        return new_id, store[new_id]["history"], store, store[new_id]["name"]
    return active, history, store, ""


def on_search(query, store):
    return gr.update(choices=radio_choices(store, query))


def on_delete(selected, store, active, history):
    if not selected:
        return store, active, history, gr.update(), ""
    del_id = label_to_id(store, selected)
    if not del_id or del_id not in store:
        return store, active, history, gr.update(), ""
    was_active = (del_id == active)
    if not was_active and active in store:
        store[active]["history"] = list(history)
    del store[del_id]
    if not store:
        cid, data = make_chat()
        store[cid] = data
        active = cid
        history = []
    elif was_active:
        active = list(store.keys())[0]
        history = store[active]["history"]
    choices = radio_choices(store)
    lbl = chat_label(store[active])
    return store, active, history, gr.update(choices=choices, value=lbl), ""


def on_rename(selected, new_name, store):
    if not selected or not new_name.strip():
        return store, gr.update(), ""
    cid = label_to_id(store, selected)
    if cid and cid in store:
        store[cid]["name"] = new_name.strip()
    choices = radio_choices(store)
    new_lbl = chat_label(store[cid]) if cid else None
    return store, gr.update(choices=choices, value=new_lbl), ""


def on_download_txt(selected, store):
    if not selected:
        return gr.update(visible=False)
    cid = label_to_id(store, selected)
    if not cid or cid not in store:
        return gr.update(visible=False)
    chat = store[cid]
    safe = re.sub(r'[^\w\s-]', '', chat['name']).strip().replace(' ', '_') or "chat"
    path = f"/tmp/{safe}_{cid}.txt"
    with open(path, "w", encoding="utf-8") as f:
        f.write(f"Chat: {chat['name']}\n")
        f.write(f"Created: {chat['created']}\n")
        f.write(f"Model: {MODEL_LABEL}\n")
        f.write("=" * 50 + "\n\n")
        for u, b in chat.get("history", []):
            f.write(f"You:\n{u}\n\n")
            f.write(f"Assistant:\n{clean_history_text(b)}\n\n")
            f.write("-" * 30 + "\n\n")
    return gr.update(value=path, visible=True)


def on_download_json(selected, store):
    if not selected:
        return gr.update(visible=False)
    cid = label_to_id(store, selected)
    if not cid or cid not in store:
        return gr.update(visible=False)
    chat = store[cid]
    safe = re.sub(r'[^\w\s-]', '', chat['name']).strip().replace(' ', '_') or "chat"
    path = f"/tmp/{safe}_{cid}.json"
    msgs = []
    for u, b in chat.get("history", []):
        msgs.append({"role": "user", "content": u})
        msgs.append({"role": "assistant", "content": clean_history_text(b)})
    export = {
        "name": chat["name"],
        "created": chat["created"],
        "model": f"{MODEL_LABEL} ({LOADED_QUANT_TAG})",
        "messages": msgs,
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(export, f, indent=2, ensure_ascii=False)
    return gr.update(value=path, visible=True)


def on_font_change(chat_size, scale):
    s = int(chat_size)
    sc = int(scale)
    factor = sc / 100
    return f"""<style>
    #chatbot .message-wrap, #chatbot .message-wrap *,
    #chatbot .prose, #chatbot .prose *,
    #chatbot p, #chatbot li, #chatbot span {{
        font-size: {s}px !important;
        line-height: {round(s * 1.55)}px !important;
    }}
    .gradio-container button {{
        font-size: {round(14 * factor)}px !important;
        padding: {round(8 * factor)}px {round(16 * factor)}px !important;
    }}
    .gradio-container input,
    .gradio-container textarea,
    .gradio-container select {{
        font-size: {round(14 * factor)}px !important;
    }}
    .gradio-container label,
    .gradio-container .label-wrap,
    .gradio-container .info {{
        font-size: {round(12 * factor)}px !important;
    }}
    .gradio-container .markdown-text,
    .gradio-container .markdown-text * {{
        font-size: {round(14 * factor)}px !important;
    }}
    .gradio-container h2 {{
        font-size: {round(24 * factor)}px !important;
    }}
    .gradio-container h3 {{
        font-size: {round(18 * factor)}px !important;
    }}
    .gradio-container sub {{
        font-size: {round(11 * factor)}px !important;
    }}
    #sidebar-col .wrap label {{
        font-size: {round(13 * factor)}px !important;
        padding: {round(8 * factor)}px {round(10 * factor)}px !important;
    }}
    #sidebar-rail button {{
        width: {round(38 * factor)}px !important;
        height: {round(38 * factor)}px !important;
        font-size: {round(16 * factor)}px !important;
    }}
    #settings-btn, #upload-btn, #collapse-btn {{
        min-width: {round(42 * factor)}px !important;
        height: {round(36 * factor)}px !important;
        font-size: {round(18 * factor)}px !important;
    }}
    </style>"""


# ══════════════════════════════════════════════════════════════
# Context Reload
# ══════════════════════════════════════════════════════════════

def reload_context(new_ctx):
    global llm, CONTEXT_LENGTH
    new_ctx = int(new_ctx)

    if new_ctx == CONTEXT_LENGTH:
        return f"ℹ️ Already at ctx={new_ctx}"

    try:
        llm.close()
    except:
        pass
    try:
        del llm
    except:
        pass
    gc.collect()

    try:
        import torch
        torch.cuda.empty_cache()
    except:
        pass
    gc.collect()

    try:
        from llama_cpp import Llama
        llm = Llama(
            model_path=LOADED_MODEL_PATH,
            n_gpu_layers=GPU_LAYERS,
            n_ctx=new_ctx,
            n_batch=384,
            use_mmap=False,
            use_mlock=False,
            verbose=False,
        )
        CONTEXT_LENGTH = new_ctx
        return f"✅ Reloaded with ctx={new_ctx}"
    except Exception as e:
        return f"❌ Failed: {e}\n\nTry a smaller context or fewer GPU layers."


# ══════════════════════════════════════════════════════════════
# Chat Function (Streaming + File Upload)
# ══════════════════════════════════════════════════════════════

def chat_fn(user_message, file_upload, history, store, active,
            system_prompt, max_tokens, temperature, top_p, min_p, repeat_penalty):

    file_context = ""
    file_notice = ""

    if file_upload is not None:
        content, ftype, fname = process_uploaded_file(file_upload)
        if ftype == "text" and content:
            file_context = f"\n\n---\n📎 **Attached file: {fname}**\n```\n{content}\n```\n---\n"
            file_notice = f"📎 *{fname}*\n\n"
        elif ftype == "image":
            file_context = (
                f"\n\n[User attached an image: {fname}. "
                f"Note: This model is text-only and cannot process images. "
                f"Please describe what you'd like help with regarding this image.]"
            )
            file_notice = f"🖼️ *{fname}* (⚠️ text-only model)\n\n"
        elif ftype == "unsupported":
            file_context = f"\n\n[User attached a file: {fname}. This file type is not supported for direct reading.]"
            file_notice = f"📄 *{fname}* (unsupported format)\n\n"

    combined_message = user_message.strip()
    if not combined_message and not file_context:
        yield "", None, history, store, gr.update()
        return

    if not combined_message and file_context:
        combined_message = "Please analyze the attached file."

    full_user_message = combined_message + file_context
    display_user_message = file_notice + combined_message

    if active in store and store[active]["name"].startswith("New Chat") and not history:
        trunc = combined_message[:30].strip()
        if len(combined_message) > 30:
            trunc += "…"
        store[active]["name"] = trunc

    messages = [{"role": "system", "content": system_prompt}]
    for u, b in history:
        messages.append({"role": "user", "content": u})
        if b:
            messages.append({"role": "assistant", "content": clean_history_text(b)})
    messages.append({"role": "user", "content": full_user_message})

    history = history + [(display_user_message, "⏳ ...")]
    yield "", None, history, store, gr.update()

    t0 = time.time()
    full_response = ""
    token_count = 0

    try:
        stream = llm.create_chat_completion(
            messages=messages,
            max_tokens=int(max_tokens),
            temperature=temperature,
            top_p=top_p,
            min_p=min_p,
            repeat_penalty=repeat_penalty,
            stream=True,
        )
        for chunk in stream:
            delta = chunk["choices"][0].get("delta", {}).get("content", "")
            if not delta:
                continue
            full_response += delta
            token_count += 1
            elapsed = time.time() - t0

            # Yield every 3 tokens to reduce scroll jank
            if token_count % 3 == 0 or "\n" in delta:
                display = format_streaming(full_response, token_count, elapsed)
                history[-1] = (display_user_message, display)
                yield "", None, history, store, gr.update()

    except Exception:
        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}<|endoftext|>\n"
        prompt += "<|assistant|>\n"
        output = llm(
            prompt,
            max_tokens=int(max_tokens),
            temperature=temperature,
            top_p=top_p,
            min_p=min_p,
            repeat_penalty=repeat_penalty,
            stop=["<|endoftext|>", "<|user|>", "<|end|>"],
        )
        full_response = output["choices"][0]["text"].strip()
        token_count = len(full_response.split())

    elapsed = time.time() - t0
    display = format_final(full_response, token_count, elapsed)
    history[-1] = (display_user_message, display)

    if active in store:
        store[active]["history"] = list(history)
    choices = radio_choices(store)
    active_lbl = chat_label(store[active]) if active in store else None
    yield "", None, history, store, gr.update(choices=choices, value=active_lbl)


# ══════════════════════════════════════════════════════════════
# Avatars
# ══════════════════════════════════════════════════════════════

def make_avatar(letter, bg_color, text_color="white", size=64):
    img = PIL.Image.new("RGB", (size, size), bg_color)
    draw = PIL.ImageDraw.Draw(img)
    try:
        font = PIL.ImageFont.truetype(
            "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", size // 2
        )
    except Exception:
        font = PIL.ImageFont.load_default()
    bbox = draw.textbbox((0, 0), letter, font=font)
    tw, th = bbox[2] - bbox[0], bbox[3] - bbox[1]
    draw.text(
        ((size - tw) / 2, (size - th) / 2 - 2), letter,
        fill=text_color, font=font,
    )
    path = f"/tmp/avatar_{letter}.png"
    img.save(path)
    return path

user_avatar = make_avatar("Y", "#2563eb")
bot_avatar  = make_avatar("G", "#7c3aed")


# ══════════════════════════════════════════════════════════════
# Gradio Layout
# ══════════════════════════════════════════════════════════════

theme = gr.themes.Soft(
    primary_hue="violet",
    secondary_hue="indigo",
    font=gr.themes.GoogleFont("Inter"),
)

CUSTOM_CSS = """
    #settings-btn, #upload-btn {
        min-width: 42px !important;
        padding: 8px !important;
    }
    #sidebar-col {
        border-right: 1px solid #e5e7eb;
        padding-right: 12px !important;
        min-width: 240px;
        transition: all 0.3s ease;
    }
    #collapse-btn {
        min-width: 36px !important;
        max-width: 36px !important;
        height: 36px !important;
        padding: 0 !important;
        border-radius: 8px !important;
        font-size: 18px !important;
        display: flex;
        align-items: center;
        justify-content: center;
    }
    #collapse-btn:hover {
        background: #ede9fe !important;
    }
    #sidebar-rail {
        min-width: 48px !important;
        max-width: 48px !important;
        border-right: 1px solid #e5e7eb;
        display: flex;
        flex-direction: column;
        align-items: center;
        padding-top: 8px !important;
        gap: 8px;
        transition: all 0.3s ease;
    }
    #sidebar-rail button {
        width: 38px !important;
        height: 38px !important;
        min-width: 38px !important;
        padding: 0 !important;
        display: flex;
        align-items: center;
        justify-content: center;
        border-radius: 8px;
        font-size: 16px;
    }
    #sidebar-rail button:hover {
        background: #ede9fe !important;
    }
    #chatbot {
        min-height: 480px;
        overflow-y: auto !important;
    }
    #sidebar-col .wrap label {
        border-radius: 8px !important;
        padding: 8px 10px !important;
        margin: 2px 0 !important;
        font-size: 13px !important;
        transition: background 0.15s;
    }
    #sidebar-col .wrap label:hover {
        background: #f5f3ff !important;
    }
    #sidebar-col .wrap input[type="radio"]:checked + label {
        background: #ede9fe !important;
        border-color: #a78bfa !important;
    }
"""

SCROLL_JS = """
<script>
(() => {
    let userScrolledUp = false;

    function setup() {
        const wrap = document.querySelector('#chatbot .wrap')
                  || document.querySelector('#chatbot > div > div');
        if (!wrap) return setTimeout(setup, 800);

        wrap.addEventListener('scroll', () => {
            const gap = wrap.scrollHeight - wrap.scrollTop - wrap.clientHeight;
            userScrolledUp = gap > 100;
        }, {passive: true});

        const mo = new MutationObserver(() => {
            if (!userScrolledUp) {
                requestAnimationFrame(() => {
                    wrap.scrollTop = wrap.scrollHeight;
                });
            }
        });
        mo.observe(wrap, {childList: true, subtree: true, characterData: true});

        try {
            const pd = Object.getOwnPropertyDescriptor(Element.prototype, 'scrollTop');
            if (pd && pd.set) {
                Object.defineProperty(wrap, 'scrollTop', {
                    get() { return pd.get.call(this); },
                    set(v) {
                        if (userScrolledUp) return;
                        pd.set.call(this, v);
                    }
                });
            }
        } catch(e) {}

        document.addEventListener('click', e => {
            const b = e.target.closest('button');
            if (b && (b.textContent.includes('Send') || b.classList.contains('primary')))
                userScrolledUp = false;
        });
        document.addEventListener('keydown', e => {
            if (e.key==='Enter' && !e.shiftKey && e.target.tagName==='TEXTAREA')
                userScrolledUp = false;
        });
    }
    setTimeout(setup, 2000);
})();
</script>
"""

with gr.Blocks(theme=theme, title=MODEL_LABEL, css=CUSTOM_CSS) as demo:

    # ── State ──
    chat_store     = gr.State(_init_store)
    active_chat_id = gr.State(_init_id)
    sidebar_state  = gr.State(True)
    settings_state = gr.State(False)

    font_css = gr.HTML(SCROLL_JS)

    gr.Markdown(
        f"## 🟣 {MODEL_LABEL}\n"
        f"<sub>GLM-4.7 Flash · Claude Opus 4.5 · XHigh Reasoning Distill"
        f" · {LOADED_QUANT_TAG} · ctx {CONTEXT_LENGTH}</sub>"
    )

    with gr.Row():

        # ════════════════════════════════════
        # COLLAPSED SIDEBAR (thin rail)
        # ════════════════════════════════════
        with gr.Column(visible=False, elem_id="sidebar-rail") as sidebar_rail:
            rail_expand_btn  = gr.Button("☰", size="sm")
            rail_newchat_btn = gr.Button("✚", size="sm")

        # ════════════════════════════════════
        # EXPANDED SIDEBAR
        # ════════════════════════════════════
        with gr.Column(scale=1, visible=True, elem_id="sidebar-col") as sidebar_panel:

            with gr.Row():
                gr.Markdown("**💬 Chats**")
                collapse_btn = gr.Button("✕", elem_id="collapse-btn", size="sm", scale=0)

            search_box = gr.Textbox(
                placeholder="🔍 Search chats…",
                show_label=False,
                container=False,
            )

            new_chat_btn = gr.Button("+ New Chat", variant="primary", size="sm")

            chat_list = gr.Radio(
                choices=radio_choices(_init_store),
                value=chat_label(_init_data),
                label="",
                interactive=True,
            )

            gr.Markdown("---")
            gr.Markdown("**Chat Actions**")

            with gr.Row():
                rename_input = gr.Textbox(
                    placeholder="New name…",
                    show_label=False,
                    container=False,
                    scale=3,
                )
                rename_btn = gr.Button("Rename", size="sm", scale=1)

            with gr.Row():
                delete_btn  = gr.Button("🗑️ Delete", size="sm", variant="stop")
                dl_txt_btn  = gr.Button("📥 TXT", size="sm")
                dl_json_btn = gr.Button("📥 JSON", size="sm")

            download_output = gr.File(
                label="Download ready", visible=False, interactive=False
            )

        # ════════════════════════════════════
        # MAIN CHAT AREA
        # ════════════════════════════════════
        with gr.Column(scale=4):

            chatbot = gr.Chatbot(
                height=520,
                show_label=False,
                bubble_full_width=False,
                avatar_images=(user_avatar, bot_avatar),
                elem_id="chatbot",
            )

            file_upload = gr.File(
                label="📎 Attach a file (text, code, csv, pdf, image)",
                file_types=[
                    ".txt", ".md", ".py", ".js", ".ts", ".html", ".css",
                    ".json", ".csv", ".xml", ".yaml", ".yml", ".log",
                    ".sh", ".sql", ".c", ".cpp", ".h", ".java", ".rs",
                    ".go", ".rb", ".php", ".r", ".tex", ".ini", ".cfg",
                    ".toml", ".env", ".pdf",
                    ".png", ".jpg", ".jpeg", ".gif", ".bmp", ".webp",
                ],
                visible=False,
            )

            upload_status = gr.Markdown("", visible=False)

            with gr.Row():
                upload_btn = gr.Button("📎", elem_id="upload-btn", scale=0)
                txt = gr.Textbox(
                    placeholder="Type your message here…",
                    show_label=False,
                    scale=8,
                    container=False,
                )
                send_btn = gr.Button("Send", variant="primary", scale=1)
                settings_toggle = gr.Button("⚙️", elem_id="settings-btn", scale=0)

            with gr.Row():
                clear_btn = gr.Button("🗑️ Clear Chat", size="sm")

        # ════════════════════════════════════
        # RIGHT SETTINGS PANEL
        # ════════════════════════════════════
        with gr.Column(scale=2, visible=False) as settings_panel:

            gr.Markdown("### ⚙️ Settings")

            system_prompt = gr.Textbox(
                label="System Prompt",
                value=DEFAULT_SYSTEM_PROMPT,
                lines=3, max_lines=6,
            )

            gr.Markdown("---")
            gr.Markdown("**Sampling**")

            temperature = gr.Slider(
                0.0, 2.0, DEFAULT_TEMPERATURE, step=0.05,
                label="Temperature",
                info="Higher = creative · Lower = focused",
            )
            max_tokens = gr.Slider(
                64, 4096, DEFAULT_MAX_TOKENS, step=64,
                label="Max Tokens",
                info="Maximum response length",
            )
            top_p = gr.Slider(
                0.0, 1.0, DEFAULT_TOP_P, step=0.01,
                label="Top-P",
                info="Nucleus sampling",
            )
            min_p = gr.Slider(
                0.0, 0.5, DEFAULT_MIN_P, step=0.005,
                label="Min-P",
                info="Use 0.01 for llama.cpp",
            )
            repeat_penalty = gr.Slider(
                1.0, 2.0, DEFAULT_REPEAT_PENALTY, step=0.05,
                label="Repeat Penalty",
                info="1.0 = disabled",
            )

            gr.Markdown("---")
            gr.Markdown("**Model**")

            ctx_dropdown = gr.Dropdown(
                choices=[2048, 4096, 8192, 16384, 32768, 65536],
                value=CONTEXT_LENGTH,
                label="Context Length",
                info="Max conversation memory (model supports up to 200K)",
            )

            ctx_reload_btn = gr.Button("🔄 Apply Context (reloads model)", size="sm")
            ctx_status = gr.Markdown("")

            gr.Markdown("---")
            gr.Markdown("**Display**")

            font_size = gr.Slider(
                10, 24, DEFAULT_FONT_SIZE, step=1,
                label="Chat Font Size (px)",
                info="Message text in chat bubbles",
            )

            ui_scale = gr.Slider(
                80, 150, DEFAULT_UI_SCALE, step=5,
                label="UI Scale (%)",
                info="Buttons, labels, inputs, sidebar — everything else",
            )

            gr.Markdown("---")
            gr.Markdown("**Presets**")
            with gr.Row():
                preset_general  = gr.Button("📝 General", size="sm")
                preset_tool     = gr.Button("🔧 Tool", size="sm")
                preset_creative = gr.Button("🎨 Creative", size="sm")

    # ══════════════════════════════════════════════════════════
    # Event Wiring
    # ══════════════════════════════════════════════════════════

    # ── Sidebar collapse / expand ──
    def do_collapse():
        return False, gr.update(visible=False), gr.update(visible=True)

    def do_expand():
        return True, gr.update(visible=True), gr.update(visible=False)

    collapse_btn.click(
        do_collapse, None,
        [sidebar_state, sidebar_panel, sidebar_rail],
    )

    rail_expand_btn.click(
        do_expand, None,
        [sidebar_state, sidebar_panel, sidebar_rail],
    )

    rail_newchat_btn.click(
        on_new_chat,
        [chat_store, active_chat_id, chatbot],
        [chat_store, active_chat_id, chatbot, chat_list, rename_input],
    ).then(
        do_expand, None,
        [sidebar_state, sidebar_panel, sidebar_rail],
    )

    # ── Settings toggle ──
    def toggle_settings(s):
        return not s, gr.update(visible=not s)

    settings_toggle.click(
        toggle_settings, [settings_state], [settings_state, settings_panel]
    )

    # ── Upload toggle ──
    upload_visible_state = gr.State(False)

    def toggle_upload(visible):
        return gr.update(visible=not visible), not visible

    upload_btn.click(
        toggle_upload,
        [upload_visible_state],
        [file_upload, upload_visible_state],
    )

    def on_file_selected(file_obj):
        if file_obj is None:
            return gr.update(visible=False, value="")
        fname = os.path.basename(file_obj.name) if hasattr(file_obj, 'name') else "file"
        ext = os.path.splitext(fname)[1].lower()
        if ext in SUPPORTED_IMG_EXT:
            return gr.update(visible=True, value=f"🖼️ **{fname}** attached (⚠️ model is text-only)")
        elif ext in SUPPORTED_TEXT_EXT or ext == ".pdf" or ext == ".csv":
            fsize = os.path.getsize(file_obj.name) / 1024
            return gr.update(visible=True, value=f"📎 **{fname}** ({fsize:.0f} KB) — will be sent with next message")
        else:
            return gr.update(visible=True, value=f"📄 **{fname}** — unsupported format")

    file_upload.change(on_file_selected, [file_upload], [upload_status])

    # ── Sidebar events ──
    search_box.change(on_search, [search_box, chat_store], [chat_list])

    new_chat_btn.click(
        on_new_chat,
        [chat_store, active_chat_id, chatbot],
        [chat_store, active_chat_id, chatbot, chat_list, rename_input],
    )

    chat_list.change(
        on_select_chat,
        [chat_list, chat_store, active_chat_id, chatbot],
        [active_chat_id, chatbot, chat_store, rename_input],
    )

    delete_btn.click(
        on_delete,
        [chat_list, chat_store, active_chat_id, chatbot],
        [chat_store, active_chat_id, chatbot, chat_list, rename_input],
    )

    rename_btn.click(
        on_rename,
        [chat_list, rename_input, chat_store],
        [chat_store, chat_list, rename_input],
    )

    dl_txt_btn.click(on_download_txt, [chat_list, chat_store], [download_output])
    dl_json_btn.click(on_download_json, [chat_list, chat_store], [download_output])

    # ── Context reload ──
    ctx_reload_btn.click(reload_context, [ctx_dropdown], [ctx_status])

    # ── Settings: font + UI scale ──
    font_size.change(on_font_change, [font_size, ui_scale], [font_css])
    ui_scale.change(on_font_change, [font_size, ui_scale], [font_css])

    # ── Settings: presets ──
    def p_general():  return 1.0, 1024, 0.95, 0.01, 1.0
    def p_tool():     return 0.7, 1024, 1.0,  0.01, 1.0
    def p_creative(): return 1.2, 2048, 0.95, 0.01, 1.0

    preset_out = [temperature, max_tokens, top_p, min_p, repeat_penalty]
    preset_general.click(p_general,   None, preset_out)
    preset_tool.click(p_tool,         None, preset_out)
    preset_creative.click(p_creative, None, preset_out)

    # ── Clear current chat ──
    def clear_current(store, active):
        if active in store:
            store[active]["history"] = []
        return [], store

    clear_btn.click(
        clear_current,
        [chat_store, active_chat_id],
        [chatbot, chat_store],
    )

    # ── Chat submit ──
    chat_inputs = [
        txt, file_upload, chatbot, chat_store, active_chat_id,
        system_prompt, max_tokens, temperature, top_p, min_p, repeat_penalty,
    ]
    chat_outputs = [txt, file_upload, chatbot, chat_store, chat_list]

    txt.submit(chat_fn, chat_inputs, chat_outputs)
    send_btn.click(chat_fn, chat_inputs, chat_outputs)

demo.launch(debug=False, share=True, quiet=True)

In [ ]:
# ============================================================
# @title 🧹 CELL 5 — Full Unload (Model + Gradio + VRAM + RAM)
# ============================================================

import gc

# ── Stop Gradio ──
try:
    demo.close()
    print("✅ Gradio stopped")
except:
    print("ℹ️ Gradio not running")

# ── Unload model ──
try:
    llm.close()
    print("✅ llm.close() done")
except:
    pass

try:
    del llm
    print("✅ llm deleted")
except:
    print("ℹ️ llm not found")

# ── Force garbage collection (multiple passes) ──
gc.collect()
gc.collect()
gc.collect()

# ── Clear CUDA cache ──
try:
    import torch
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("✅ CUDA cache cleared")
except:
    pass

gc.collect()

# ── Verify ──
import psutil, subprocess

ram_used = psutil.virtual_memory().used / 1e9
try:
    smi = subprocess.check_output(
        ["nvidia-smi", "--query-gpu=memory.used,memory.free",
         "--format=csv,noheader,nounits"], text=True
    ).strip().split(", ")
    print(f"\n📊 VRAM: {int(smi[0])} MiB used / {int(smi[1])} MiB free")
except:
    pass

print(f"📊 RAM:  {ram_used:.1f} GB used")
print("\n🧹 Cleanup complete — safe to reload or switch models")
print("   Run Cell 2 → Cell 3 → Cell 4 to load a different model")